In [1]:
# Import necessary libraries

import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as bs4

Scrape Data from [Cars45](https://www.cars45.com)

In [7]:
# Get links for each car
base_url = "https://www.cars45.com"
urls = []

for i in tqdm(range(1, 201)):
    req = requests.get(f"{base_url}/listing?page={i}").text
    data = bs4(req, 'html.parser')
    for item in data.find_all(class_='car-feature'):
        urls.append(item['href'])

100%|█████████████████████████████████████████| 200/200 [03:54<00:00,  1.17s/it]


In [8]:
# Get car information
make = []
model = []
year = []
color = []
condition = []
mileage = []
transmission = []
price = []

link_with_errors = []

for link in tqdm(urls):
    try:
        req = requests.get(f"{base_url}{link}").text
        data = bs4(req, 'html.parser')

        price.append(data.find("h5", class_="main-details__name__price").text.strip().replace("₦", "").lstrip())
        transmission.append(data.find("div", class_="main-details__tags flex wrap").find_all("span")[1].text.strip())

        description = [item.text.strip() for item in data.find_all("p", class_="general-info__name")]
        make.append(description[0])
        model.append(description[1])
        year.append(description[2])
        color.append(description[3])
        condition.append(description[4])
        mileage.append(description[5])
    except:
        link_with_errors.append(link)
        print(f"Error occured while scraping data: {base_url}{link}")

100%|███████████████████████████████████████| 3000/3000 [36:36<00:00,  1.37it/s]


In [9]:
# Put each array into a dataFrame
data = {"Make": make, "Model": model, "Year": year, "Color": color, "Condition": condition, "Mileage": mileage, "Transmission": transmission, "Price": price}
car_data = pd.DataFrame({k: pd.Series(v) for k,v in data.items()})
car_data

,Make,Model,Year,Color,Condition,Mileage,Transmission,Price
0,Honda,Accord,2003,Gray,Nigerian Used,219081,Automatic,"1,800,000"
1,Toyota,Highlander,2008,Blue,Foreign Used,168287,Automatic,"11,250,000"
2,Volkswagen,Passat,2008,Blue,Nigerian Used,207086,Automatic,"1,864,687"
3,Hyundai,Santa Fe,2003,Black,Nigerian Used,220459,Automatic,"1,687,500"
4,Audi,Q5,2010,3.2 Premium Quattro,White,Nigerian Used,Automatic,"5,625,000"
...,...,...,...,...,...,...,...,...
2995,Toyota,Camry,2006,Gray,Nigerian Used,392464,Automatic,"2,475,000"
2996,Lexus,RX 350,2012,White,Nigerian Used,86028,Automatic,"10,400,000"
2997,Toyota,Avalon,2006,Gold,Nigerian Used,76211,Automatic,"3,037,500"
2998,Toyota,Hilux,2021,Black,Nigerian Used,2027,Automatic,"50,625,000"


In [10]:
# Save DataFrame to csv
car_data.to_csv("data/car-listing-data.csv", index=False)